In [1]:
%run D:\qtlab_replacement\init.py
import matplotlib.pyplot as plt
from instruments import *
import numpy as np
import sweep
from save_pkl import *
%matplotlib qt5
from matplotlib import interactive
interactive(True)

Failed loading module Signal_Hound_SA:  [WinError 126] The specified module could not be found


In [2]:
current_src1 = Yokogawa_GS210(address='GPIB::1::INSTR')
current_src2 = Yokogawa_GS210(address='GPIB::2::INSTR')

#current_src = Yokogawa_GS210(address='USB0::0x0B21::0x0039::91T112494::INSTR')
#current_src2 = Yokogawa_GS210(address='USB0::0x0B21::0x0039::91T112492::INSTR')

In [3]:
import ftd2xx 
print(ftd2xx.listDevices())
awg_sema0 = AWG500('awg',address=0, ver=2)

None


DeviceError: DEVICE_NOT_FOUND

In [3]:
#current_src = Yokogawa_GS210(address='gs210')
pna = Agilent_N5242A('pna', address = 'TCPIP0::A-N5242A-22023::inst0::INSTR')
lo1 = Agilent_E8257D('lo1', address = 'TCPIP0::10.20.61.59::inst0::INSTR')
#pxa = Agilent_N9030A('pxa', address = 'PXA')
#awg_tek = Tektronix_AWG5014('awg_tek', address = 'TCPIP0::10.20.61.186::inst0::INSTR')
adc = Spectrum_M3i2132('adc')
sa = Agilent_N9030A('pxa', address = 'TCPIP0::10.20.61.56::inst0::INSTR')
#adc = Spectrum_M3i2132('adc')
# векторный анализатор в режим линейного свипа
pna.set_sweep_mode("LIN")
#dac=nndac()

True

In [4]:
dac=nndac()

In [5]:
def set_res_z_voltage(dc):
    #if dc<-1.0:
    #    dc=-1.0
    #if dc>1.0:
    #    dc=1.0
    #waveform = np.ones(awg_sema0.get_nop()+200)*dc
    #print(dc)
    awg_tek.set_offset(dc, channel=2)
    #awg_sema0.set_waveform(waveform, channel=0)
    #awg_tek.set_status(1, channel=4)
    #awg_sema0.run()

In [11]:
awg_sema0.set_repetition_period(20e-6)

True

# Spectroscopy with passthrough mixers

In [95]:
# AWG для того чтобы открыть смесители
pna.set_sweep_mode("LIN")
awg_tek = Tektronix_AWG5014('awg_tek', address = 'TCPIP0::10.20.61.186::inst0::INSTR')
Vdc_q = 0.1
Vdc_r = 0.1
# Задаём постояннmtые смещения на каналы 1 и 3
awg_tek.set_waveform([0]*awg_tek.get_nop(), channel=1)
awg_tek.set_waveform([0]*awg_tek.get_nop(), channel=2)
awg_tek.set_waveform([0]*awg_tek.get_nop(), channel=3)
awg_tek.set_waveform([0]*awg_tek.get_nop(), channel=4)
awg_tek.set_offset(Vdc_q, channel=1)
awg_tek.set_offset(Vdc_q, channel=2)
awg_tek.set_offset(Vdc_r, channel=3)
awg_tek.set_offset(Vdc_r, channel=4)

# Сами импульсы надо при этом выключить! Будет меньше шуметь (но то не точно).
awg_tek.stop()

root: WARNING  Tektronix_AWG5014 : get all not yet fully functional


# Resonator tunability spectorscopy

In [5]:
import time
class zero_device:
    def get_points(self):
        return {'dummy':[('dummy', [0])]}
    def get_dtype(self):
        return {'dummy':np.float}
    def get_opts(self):
        return {'dummy': {'log': None}}
    def measure(self):
        time.sleep(100)
        return {'dummy':[0.0]}
zd = zero_device()

In [9]:
voltages = np.linspace(-2.5, +2.5, 51)
channel_ids = range(0,24)
for channel_id in channel_ids:
    for v in voltages:
        dac.set_voltage(channel_id,v)
        time.sleep(10.)
    dac.set_voltage(channel_id,0.)

NameError: name 'dac' is not defined

In [10]:
dac.close()

NameError: name 'dac' is not defined

In [12]:
channel_ids = range(0,24)
for channel_id in channel_ids:
    print (dac.get_voltage(channel_id))

VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.

In [8]:
#Включаем всё что надо и выключаем всё что не надо
awg_ch_on = [0]
awg_ch_off = [ch_id for ch_id in range(7) if ch_id not in awg_ch_on]

for ch_id in awg_ch_off:
    ch_settings = awg_sema0.get_channel_settings(ch_id)
    ch_settings['on'] = False
    awg_sema0.set_channel_settings(ch_id, ch_settings)

for ch_id in awg_ch_on:
    ch_settings = awg_sema0.get_channel_settings(ch_id)
    ch_settings['on'] = True
    ch_settings['slow'] = 0
    ch_settings['mode'] = 'ram'
    ch_settings['delay'] = 0
    awg_sema0.set_channel_settings(ch_id, ch_settings)
#хз что
awg_sema0.ProgTimer(awg_sema0.intToBytes([0xfc, 0]))
#1 триггер на один период сигнала
awg_sema0.set_trigger_repeats(1)
#Выход триггера
awg_sema0.set_trigger_pulse("SYNC0", awg_sema0.create_delayed_trigger_pulse(100.e-9) )
awg_sema0.set_trigger_pulse("SYNC3", awg_sema0.create_delayed_trigger_pulse(100.e-9) )

awg_sema0.set_nop(128)

In [13]:
pna.set_power(-55)
pna.set_bandwidth(1)

True

In [7]:
print('Power: {0} dBm'.format(pna.get_power()))
print('BW: {0} Hz'.format(pna.get_bandwidth()))
print('Frequency limits [{0}, {1}] Hz, number of points: {2}'.format(pna.get_xlim()[0], pna.get_xlim()[1], pna.get_nop()))
plt.plot(pna.get_freqpoints(), pna.get_tracedata()[0])

Power: -35.0 dBm
BW: 1000.0 Hz
Frequency limits [7000000000.0, 10000000000.0] Hz, number of points: 2001


In [ ]:
voltages = np.linspace(-2.5, +2.5, 51)
channel_ids = range(0,24)
for channel_id in channel_ids:
    set_channel_voltage = lambda x: dac.set_voltage(channel_id,x)
    sweep.sweep(zd, (voltages,
                      set_channel_voltage, 
                      'Channel {0} voltage'.format(channel_id)), 
                filename='Zeros NNDAC channel {0}'.format(channel_id), output=False)
    dac.set_voltage(channel_id,0.)

In [ ]:
voltages = np.linspace(-2.5, +2.5, 51)
channel_ids = range(0,24,3)
for channel_id in channel_ids:
    set_channel_voltage = lambda x: dac.set_voltage(channel_id,x)
    sweep.sweep(pna, (voltages,
                      set_channel_voltage, 
                      'Channel {0} voltage'.format(channel_id)), 
                filename='Hivemind NNDAC channel {0}'.format(channel_id))
    dac.set_voltage(channel_id,0.)

In [5]:
for channel_id in range(24):
    dac.set_voltage(channel_id, 0)
dac.get_voltage(0), dac.get_voltage(1), dac.get_voltage(2)

('0.000', '0.000', '0.000')

In [ ]:
currents = np.linspace(-0.12, 0.12, 601)
#current_src.set_status(0)
current_src1.set_status(1)
#sweep.sweep(pna, (currents, dac.set_voltage, 'Coil current'), filename='Hivemind coil 1')
sweep.sweep(pna, (currents, current_src1.set_current, 'Coil current'), filename='Hivemind coil res-z')

A Jupyter Widget

Started at:  May 10 2018 19:59:25


KeyboardInterrupt: 

tornado.general: ERROR    Uncaught exception, closing connection.
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\zmq\eventloop\zmqstream.py", line 414, in _run_callback
    callback(*args, **kwargs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\tornado\stack_context.py", line 277, in null_wrapper
    return fn(*args, **kwargs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\kernelbase.py", line 235, in dispatch_shell
    handler(stream, idents, msg)
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\kernelbase.py", line 421, in execute_request
    self._abort_queues()
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\kernelbase.py", line 637, in _abort_queues
    self._abort_queue(stream)
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\kernelbase.py", li

In [10]:
current_src.get_voltage_compliance()

8.0

In [5]:
pna.set_xlim(6e9, 10e9)
pna.set_nop(401)
pna.set_power(-35)
pna.set_bandwidth(1000)
#lo1.set_power(-15)
#freqs = np.linspace(6e9, 6.7e9, 601)
currents = np.linspace(-2e-3, 2e-3, 101)
lo1.set_status(0)
current_src1.set_status(1)
sweep.sweep(pna, (currents, current_src1.set_current, 'Coil current'), filename='Hivemind #2 single-tone antenna')

A Jupyter Widget

Started at:  May 10 2018 11:01:14

Elapsed time: 0 h 1 m 13.7 s


{'S-parameter': (('Coil current', 'Frequency'),
  (array([ -2.00000000e-03,  -1.96000000e-03,  -1.92000000e-03,
           -1.88000000e-03,  -1.84000000e-03,  -1.80000000e-03,
           -1.76000000e-03,  -1.72000000e-03,  -1.68000000e-03,
           -1.64000000e-03,  -1.60000000e-03,  -1.56000000e-03,
           -1.52000000e-03,  -1.48000000e-03,  -1.44000000e-03,
           -1.40000000e-03,  -1.36000000e-03,  -1.32000000e-03,
           -1.28000000e-03,  -1.24000000e-03,  -1.20000000e-03,
           -1.16000000e-03,  -1.12000000e-03,  -1.08000000e-03,
           -1.04000000e-03,  -1.00000000e-03,  -9.60000000e-04,
           -9.20000000e-04,  -8.80000000e-04,  -8.40000000e-04,
           -8.00000000e-04,  -7.60000000e-04,  -7.20000000e-04,
           -6.80000000e-04,  -6.40000000e-04,  -6.00000000e-04,
           -5.60000000e-04,  -5.20000000e-04,  -4.80000000e-04,
           -4.40000000e-04,  -4.00000000e-04,  -3.60000000e-04,
           -3.20000000e-04,  -2.80000000e-04,  -2.400000

In [145]:
set_res_z_voltage(-0)

0


In [17]:
lo1.set_status(1)
#lo1_powers = np.linspace(-20, 0, 21)
lo1.set_power(0)
lo1_freqs = np.linspace(3e9, 12e9, 9001)
pna.set_power(-30)
pna.set_xlim(10.06e9, 10.06e9)
pna.set_nop(1)
sweep.sweep(pna, (lo1_freqs, lo1.set_frequency, 'Second tone frequency'), filename='Hivemind two-tone (x)')

Started at:  Jan 24 2018 08:32:30

Elapsed time: 1 h 8 m 29.23 s


{'S-parameter': (('Second tone frequency',),
  (array([  3.00000000e+09,   3.00100000e+09,   3.00200000e+09, ...,
            1.19980000e+10,   1.19990000e+10,   1.20000000e+10]),),
  array([-0.01623961-0.01522938j, -0.01630909-0.01537232j,
         -0.01621177-0.01546274j, ..., -0.01496260-0.01713449j,
         -0.01497149-0.01715337j, -0.01483067-0.01691502j]),
  {'log': 20},
  ('',))}

In [13]:
pna.set_xlim(10e9, 10.1e9)
pna.set_nop(101)
powers = np.linspace(-50, 0, 51)
sweep.sweep(pna, (powers, pna.set_power, 'Power'), filename='Hivemind power spectrum (x)')

Started at:  Jan 24 2018 04:51:13

Elapsed time: 0 h 0 m 46.23 s


{'S-parameter': (('Power', 'Frequency'),
  (array([-50., -49., -48., -47., -46., -45., -44., -43., -42., -41., -40.,
          -39., -38., -37., -36., -35., -34., -33., -32., -31., -30., -29.,
          -28., -27., -26., -25., -24., -23., -22., -21., -20., -19., -18.,
          -17., -16., -15., -14., -13., -12., -11., -10.,  -9.,  -8.,  -7.,
           -6.,  -5.,  -4.,  -3.,  -2.,  -1.,   0.]),
   array([  1.00000000e+10,   1.00010000e+10,   1.00020000e+10,
            1.00030000e+10,   1.00040000e+10,   1.00050000e+10,
            1.00060000e+10,   1.00070000e+10,   1.00080000e+10,
            1.00090000e+10,   1.00100000e+10,   1.00110000e+10,
            1.00120000e+10,   1.00130000e+10,   1.00140000e+10,
            1.00150000e+10,   1.00160000e+10,   1.00170000e+10,
            1.00180000e+10,   1.00190000e+10,   1.00200000e+10,
            1.00210000e+10,   1.00220000e+10,   1.00230000e+10,
            1.00240000e+10,   1.00250000e+10,   1.00260000e+10,
            1.00270000e+1

In [6]:
pna.set_xlim(9.465e9, 9.465e9)
pna.set_nop(1)
pna.set_power(-50)
pna.set_bandwidth(10)
lo1.set_power(0)
freqs = np.linspace(6.1e9, 6.5e9, 201)
currents = np.linspace(-1e-3, 1e-3, 101)
sweep.sweep(pna, (currents, current_src.set_current, 'Control current'), 
            (freqs, lo1.set_frequency, 'Second tone frequncy'), 
            filename='Hivemind two-tone korenkov')

A Jupyter Widget

Started at:  Jan 31 2018 15:59:24


Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backends\backend_qt5agg.py", line 149, in __draw_idle_agg
    self.draw()
  File "C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backends\backend_qt5agg.py", line 127, in draw
    super(FigureCanvasQTAggBase, self).draw()
  File "C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backends\backend_agg.py", line 430, in draw
    self.figure.draw(self.renderer)
  File "C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\artist.py", line 55, in draw_wrapper
    return draw(artist, renderer, *args, **kwargs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\figure.py", line 1295, in draw
    renderer, self, artists, self.suppressComposite)
  File "C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\image.py", line 138, in _draw_list_compositing_images
    a.draw(renderer)
  File "C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\artist.py", line 55, in draw_wrapper



Elapsed time: 1 h 50 m 24.33 s


{'S-parameter': (('Control current', 'Second tone frequncy'),
  (array([ -1.00000000e-03,  -9.80000000e-04,  -9.60000000e-04,
           -9.40000000e-04,  -9.20000000e-04,  -9.00000000e-04,
           -8.80000000e-04,  -8.60000000e-04,  -8.40000000e-04,
           -8.20000000e-04,  -8.00000000e-04,  -7.80000000e-04,
           -7.60000000e-04,  -7.40000000e-04,  -7.20000000e-04,
           -7.00000000e-04,  -6.80000000e-04,  -6.60000000e-04,
           -6.40000000e-04,  -6.20000000e-04,  -6.00000000e-04,
           -5.80000000e-04,  -5.60000000e-04,  -5.40000000e-04,
           -5.20000000e-04,  -5.00000000e-04,  -4.80000000e-04,
           -4.60000000e-04,  -4.40000000e-04,  -4.20000000e-04,
           -4.00000000e-04,  -3.80000000e-04,  -3.60000000e-04,
           -3.40000000e-04,  -3.20000000e-04,  -3.00000000e-04,
           -2.80000000e-04,  -2.60000000e-04,  -2.40000000e-04,
           -2.20000000e-04,  -2.00000000e-04,  -1.80000000e-04,
           -1.60000000e-04,  -1.40000000e-

In [20]:
pna.set_xlim(7.69e9, 7.69e9)
pna.set_nop(1)
pna.set_power(-45)
pna.set_bandwidth(10)
lo1.set_power(-15)
freqs = np.linspace(6e9, 6.7e9, 601)
currents = np.linspace(-0.00, 0.020, 21)
sweep.sweep(pna, (currents, set_res_z_voltage, 'Control voltage'), 
            (freqs, lo1.set_frequency, 'Second tone frequncy'), 
            filename='Hivemind two-tone res1')

Started at:  Jan 22 2018 12:48:18

Elapsed time: 2 h 31 m 43.0 s


{'S-parameter': (('Control voltage', 'Second tone frequncy'),
  (array([ 0.   ,  0.001,  0.002,  0.003,  0.004,  0.005,  0.006,  0.007,
           0.008,  0.009,  0.01 ,  0.011,  0.012,  0.013,  0.014,  0.015,
           0.016,  0.017,  0.018,  0.019,  0.02 ]),
   array([  3.00000000e+09,   3.01000000e+09,   3.02000000e+09,
            3.03000000e+09,   3.04000000e+09,   3.05000000e+09,
            3.06000000e+09,   3.07000000e+09,   3.08000000e+09,
            3.09000000e+09,   3.10000000e+09,   3.11000000e+09,
            3.12000000e+09,   3.13000000e+09,   3.14000000e+09,
            3.15000000e+09,   3.16000000e+09,   3.17000000e+09,
            3.18000000e+09,   3.19000000e+09,   3.20000000e+09,
            3.21000000e+09,   3.22000000e+09,   3.23000000e+09,
            3.24000000e+09,   3.25000000e+09,   3.26000000e+09,
            3.27000000e+09,   3.28000000e+09,   3.29000000e+09,
            3.30000000e+09,   3.31000000e+09,   3.32000000e+09,
            3.33000000e+09,   3.34

In [8]:
plt.figure('Diff 2')
plt.pcolormesh(measurement['S-parameter'][1][0], measurement['S-parameter'][1][1], np.abs(measurement['S-parameter'][2]-np.mean(measurement['S-parameter'][2], axis=0)).T)
plt.grid(True)

In [55]:
#pna.set_xlim(9.479e9, 9.479e9)
#pna.set_nop(1)
pna.set_power(-45)
pna.set_bandwidth(10)
#lo1.set_power(5)
#freqs = np.linspace(6.5e9, 9e9, 251)
voltages = np.linspace(-4, 4, 101)
for channel_id in [0,1,2,3,4,5,6,7,8,9,10,11,12,13,15,16,17,18,19,20,21,22,23,24]:
    voltage_setter = lambda x: dac.set_voltage(channel_id, x);
    sweep.sweep(pna, 
            (voltages, voltage_setter, 'Coil dac voltage #{0}'.format(channel_id)),
            (freqs, lo1.set_frequency, 'Second tone frequncy'), 
            filename='Hivemind two-tone res2 coil #{0}'.format(channel_id))
    dac.set_voltage(channel_id, 0.)

Started at:  Jan 23 2018 14:37:22

Elapsed time: 1 h 20 m 34.04 s


Started at:  Jan 23 2018 15:57:58

Elapsed time: 1 h 23 m 30.08 s


Started at:  Jan 23 2018 17:21:30

Elapsed time: 1 h 21 m 17.98 s


Started at:  Jan 23 2018 18:42:50

Elapsed time: 1 h 17 m 1.55 s


Started at:  Jan 23 2018 19:59:53

Elapsed time: 1 h 22 m 55.85 s


...

Started at:  Jan 24 2018 02:42:09

Elapsed time: 1 h 19 m 22.06 s


Started at:  Jan 24 2018 04:01:33


KeyboardInterrupt: 

KeyboardInterrupt: 

RuntimeError: You must first define an image, e.g., with imshow

In [44]:
current_src.get_status()

0

In [8]:
awg_tek.set_amplitude(0.5, channel=1)
voltages = np.linspace(-0.00, 0.020, 41)
pna.set_xlim(7.0e9,10e9)
pna.set_nop(601)
pna.set_bandwidth(500)
pna.set_power(-40)
sweep.sweep(pna, (voltages, set_res_z_voltage, 'Control voltage'), filename='Hivemind spectrum res-z_Run1')

Started at:  Jan 22 2018 12:11:59

Elapsed time: 0 h 1 m 7.71 s


{'S-parameter': (('Control voltage', 'Frequency'),
  (array([ 0.    ,  0.0005,  0.001 ,  0.0015,  0.002 ,  0.0025,  0.003 ,
           0.0035,  0.004 ,  0.0045,  0.005 ,  0.0055,  0.006 ,  0.0065,
           0.007 ,  0.0075,  0.008 ,  0.0085,  0.009 ,  0.0095,  0.01  ,
           0.0105,  0.011 ,  0.0115,  0.012 ,  0.0125,  0.013 ,  0.0135,
           0.014 ,  0.0145,  0.015 ,  0.0155,  0.016 ,  0.0165,  0.017 ,
           0.0175,  0.018 ,  0.0185,  0.019 ,  0.0195,  0.02  ]),
   array([  7.00000000e+09,   7.00500000e+09,   7.01000000e+09,
            7.01500000e+09,   7.02000000e+09,   7.02500000e+09,
            7.03000000e+09,   7.03500000e+09,   7.04000000e+09,
            7.04500000e+09,   7.05000000e+09,   7.05500000e+09,
            7.06000000e+09,   7.06500000e+09,   7.07000000e+09,
            7.07500000e+09,   7.08000000e+09,   7.08500000e+09,
            7.09000000e+09,   7.09500000e+09,   7.10000000e+09,
            7.10500000e+09,   7.11000000e+09,   7.11500000e+09,
      

In [64]:
def set_current_compensated(current_value):
    import time
    #set_res_z_voltage(-10*current_value-0.12)
    
    #current_phase = 1/(4.7e-3/4)*current_value # 4.8 half-periods in a 2mA frame => 1.2mA per period 
    #phase_delta = (current_phase+0.5)%1-0.50
    #voltage_phase = -1.95*phase_delta/0.715 # 1V per half-period => 2V per period
    
    #voltage_phase = -1.95*phase_delta/0.71 # 1V per half-period => 2V per period
    print('I1 = '+str(current_value))
    print('I2 = '+str(current_value*(-31.775)))
    #print('phase delta = '+str(phase_delta))
    #print('voltage phase = '+str(voltage_phase))
    #set_res_z_voltage(voltage_phase)
    current_src.set_current(current_value-0.00025)
    current_src2.set_current((current_value)*(-31.775))
    time.sleep(0.1)

# Inductance matrix assembly

## measure reference spectrum

In [20]:
pna.set_xlim(5.5e9, 10.5e9)
pna.set_nop(501)
pna.set_power(-20)
pna.set_bandwidth(1000)
#lo1.set_power(-15)
#freqs = np.linspace(6e9, 6.7e9, 601)
currents = np.linspace(-3e-3, 7e-3, 101)
current_src1.set_status(1)
resonator_reference_spectrum = sweep.sweep(pna, (currents, current_src1.set_current, 'Reference current', 'A'), 
            filename='Hivemind #2 single-tone resonator-z-spectrum Input thru res-z power {}'.format(int(pna.get_power())))
#plt.plot(np.angle(measurement['S-parameter'][2]))
#set_res_z_voltage(0)
current_src1.set_current(0)
current_src1.set_status(0)

A Jupyter Widget

Started at:  May 21 2018 14:58:21

Elapsed time: 0 h 1 m 27.16 s


True

In [442]:
pna.set_xlim(5e9, 7e9)
pna.set_nop(201)
pna.set_power(-50)
pna.set_bandwidth(100)
#lo1.set_power(-15)
#freqs = np.linspace(6e9, 6.7e9, 601)
currents = np.linspace(0.005, 0.015, 201)
current_src1.set_status(1)
resonator_reference_spectrum = sweep.sweep(pna, (currents, current_src1.set_current, 'Reference current', 'A'), 
            filename='Hivemind #2 single-tone resonator-z-spectrum Input thru res-z power {}'.format(int(pna.get_power())))
#plt.plot(np.angle(measurement['S-parameter'][2]))
#set_res_z_voltage(0)
current_src1.set_current(0)
current_src1.set_status(0)

A Jupyter Widget

Started at:  May 16 2018 20:20:00

Elapsed time: 0 h 6 m 57.87 s


True

In [160]:
lo1.set_status(0)

True

### load reference spectrum

In [7]:
resonator_reference_spectrum = load_pkl('20-09-28-Hivemind #2 single-tone resonator-z-spectrum Input res-x power --40\\S-parameter Hivemind #2 single-tone resonator-z-spectrum Input res-x power --40', 
                                        location='D:\\qtlab\\ReiData\\data\\2018-05-21')[1]

D:\qtlab\ReiData\data\2018-05-21/20-09-28-Hivemind #2 single-tone resonator-z-spectrum Input res-x power --40\S-parameter Hivemind #2 single-tone resonator-z-spectrum Input res-x power --40.pkl


## spectrum image preprocessing

In [8]:
from scipy.signal import medfilt2d

ref_S21u = np.unwrap(np.angle(np.mean(resonator_reference_spectrum['S-parameter'][2], axis=0)))
phase_min = ref_S21u[0]
phase_max = ref_S21u[-1]
def resonator_spectrum_preprocessing(x):
    S21d = x*np.exp(-np.reshape(np.linspace(phase_min,phase_max,ref_S21u.shape[0]), (1,-1))*1j)
    S21d = np.gradient(S21d)[1]
    return S21d
    #inp = S21d-medfilt2d(np.real(S21d),3)-1j*medfilt2d(np.imag(S21d),3)
    
    return inp

## find centre&period of spectrum

In [11]:
reference_preprocessed = resonator_spectrum_preprocessing(resonator_reference_spectrum['S-parameter'][2])
centre_reference_pixels, period_reference_pixels = fitting.xcorr_centre_period(reference_preprocessed, axis=1)

centre_reference = resonator_reference_spectrum['S-parameter'][1][0][int(centre_reference_pixels)], 
period_reference = (resonator_reference_spectrum['S-parameter'][1][0][1]-resonator_reference_spectrum['S-parameter'][1][0][0])*period_reference_pixels
print (centre_reference, period_reference)

(0.0010699999999999998,) 0.00672


## measure spectra for other controls

In [10]:
import fitting
import imp
import instruments.nndac
imp.reload(instruments.nndac)
imp.reload(fitting)
dac = instruments.nndac.nndac()
pna.set_bandwidth(1000)

True

In [12]:
#pna.set_xlim(9.479e9, 9.479e9)
#pna.set_nop(1)
#pna.set_power(-55)
#pna.set_bandwidth(100)
#lo1.set_power(5)
#freqs = np.linspace(6.5e9, 9e9, 251)
voltages = np.linspace(-4, 4, 101)
hivemind_qubit_controls = {}
current_src1.set_status(1)
#current_src1.set_current(centre_reference[0])
current_src1.set_current(0)
for channel_id in [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]:
#for channel_id in [11,12,13,15,16,17,18,21,22,23]:
#for channel_id in [4]:
    voltage_setter = lambda x: dac.set_voltage(channel_id, x);
    hivemind_qubit_controls[channel_id] = {'resonator_spectrum':sweep.sweep(pna, 
            (voltages, voltage_setter, 'Coil dac voltage #{0}'.format(channel_id)),
            filename='Hivemind #2 single-tone res2 coil #{0}'.format(channel_id))}
    #hivemind_qubit_controls[channel_id]['preprocessed_spectrum'] = \
    #    resonator_spectrum_preprocessing(hivemind_qubit_controls[channel_id]['resonator_spectrum']['S-parameter'][2])
    dac.set_voltage(channel_id, 0.)
    
save_pkl({'type': 'Crosstalk DC', 'name': '100500 resonator-qubit controls'}, 
                      hivemind_qubit_controls, 
                      location='D:\\qtlab\\ReiData\\data\\calibrations', 
                      time=False, 
                      plot=False, 
                      matlab=False)

A Jupyter Widget

Started at:  May 21 2018 20:21:26

Elapsed time: 0 h 1 m 28.8 s


A Jupyter Widget

Started at:  May 21 2018 20:22:57

Elapsed time: 0 h 1 m 27.05 s


A Jupyter Widget

Started at:  May 21 2018 20:24:25

Elapsed time: 0 h 1 m 26.35 s


A Jupyter Widget

Started at:  May 21 2018 20:25:53


KeyboardInterrupt: 

In [17]:
dac = nndac()

### load spectra for other controls

In [230]:
hivemind_qubit_controls = load_pkl('Crosstalk DC 100500 resonator-qubit controls', 
                      location='D:\\qtlab\\ReiData\\data\\calibrations')[1]

D:\qtlab\ReiData\data\calibrations/Crosstalk DC 100500 resonator-qubit controls.pkl


In [13]:
for spectrum_id in hivemind_qubit_controls.keys():
    preprocessed_spectrum = resonator_spectrum_preprocessing(hivemind_qubit_controls[spectrum_id]['resonator_spectrum']['S-parameter'][2])
    hivemind_qubit_controls[spectrum_id]['preprocessed_spectrum'] = preprocessed_spectrum

In [256]:
hivemind_qubit_controls[0],hivemind_qubit_controls[23]
plt.figure()
plt.pcolormesh(np.real(hivemind_qubit_controls[0]['preprocessed_spectrum']))
plt.colorbar()
plt.figure()
plt.pcolormesh(np.real(hivemind_qubit_controls[23]['preprocessed_spectrum']))
plt.colorbar()

## correlate spectra for reference & other controls

In [14]:
import fitting
import imp
imp.reload(fitting)

<module 'fitting' from 'D:\\qtlab_replacement\\scripts\\fitting.py'>

In [15]:
for control_id in [i for i in hivemind_qubit_controls.keys()]:
    reference_scale_raw, max_d  = fitting.xcorr_scale(reference_preprocessed, 
                                          hivemind_qubit_controls[control_id]['preprocessed_spectrum'], 
                                          thresh=0.995,
                                          centre_reference_pixels = 0.0,
                                          max_scale = 1.)
    range_reference = (resonator_reference_spectrum['S-parameter'][1][0][-1]-resonator_reference_spectrum['S-parameter'][1][0][0])
    range_qubit_control = (hivemind_qubit_controls[control_id]['resonator_spectrum']['S-parameter'][1][0][-1]-\
                       hivemind_qubit_controls[control_id]['resonator_spectrum']['S-parameter'][1][0][0])
    Lr = -reference_scale_raw*range_reference/range_qubit_control/period_reference
    hivemind_qubit_controls[control_id]['Lr'] = Lr
    print('control_id: ', Lr, reference_scale_raw, max_d)

control_id:  0.0115039679774 -0.0515377765389 -1.42032865611
control_id:  0.0189241368829 -0.0847801332354 -3.80105180269
control_id:  -0.0245989409905 0.110203255637 -4.82235444578


## Optimizing iductance via some resonance

In [332]:
import data_reduce

In [ ]:
reference_measurer = pna
reference_measurement_name = 'S-parameter'
# now we measure the same thing and make sure it doesn't walk away
initial_offset = 0.01
final_offset = 4.0
initial_step = scale_initial*2
scale_scan_steps = 31

def set_control_compensated(x, target, compensator, scale):
    compensation = -x*scale
    target(x)
    compensator(compensation)
    #print (x, scale, compensation)

control = lambda y: dac.set_voltage(channel_id, y)
compensator = current_src1.set_current
scale_initial = hivemind_qubit_controls[channel_id]['Lr']*period_reference
    
set_control_compensated(0, control, compensator, scale_initial)
reference = pna.measure()[reference_measurement_name]
reference = reference-np.mean(reference)
pna_reducer = data_reduce.data_reduce(pna)
pna_reducer.filters['Reference feature'] = data_reduce.feature_reducer(reference_measurer, 
                                                     reference_measurement_name, 
                                                     0, 
                                                     np.zeros(reference.shape), 
                                                     np.conj(reference))
pna_reducer.filters['Reference feature']['get_opts'] = lambda: {}
#lambda: reference-pna.measure()[reference_measurement_name][2]
#sweep.optimize(target, )
num_offsets = int(np.log2(final_offset/initial_offset))
offsets = np.logspace(initial_offset, final_offset, num_offsets)
print (offsets)

step = initial_step
best_scale = scale_initial
for offset in offsets:
    set_new_scale = lambda x: set_control_compensated(offset, control, compensator, x)
    scales = np.linspace(best_scale-step, best_scale+step, scale_scan_steps)
    measurement = sweep.sweep(pna_reducer, (scales, set_new_scale, 'Compensator scale'), save=False, output=False)
    measurement1 = sweep.sweep(pna, (scales, set_new_scale, 'Compensator scale'), save=False, output=False)
    print (np.argmax(measurement['Reference feature'][2]), measurement['Reference feature'][1][0][np.argmax(measurement['Reference feature'][2])])
    best_scale = measurement['Reference feature'][1][0][np.argmax(np.real(measurement['Reference feature'][2]))]
    step = step/np.sqrt(2)

In [345]:
measurement['Reference feature'][2]

array([ 0.22675345-0.03241033j,  0.22434938-0.06446877j,
        0.28845743-0.09264065j,  0.36287219-0.03521849j,
        0.34172259+0.04679589j,  0.27899263+0.07926115j,
        0.22736187+0.06821513j])

In [328]:
sweep.optimize?

In [326]:
hivemind_qubit_controls[channel_id]['Lr']*period_reference

0.0021195033308848495

## Compensated spectrum

In [61]:
dac = nndac()

In [401]:
def set_control_compensated(x, target, compensator, scale):
    compensation = -x*scale
    target(x)
    compensator(compensation)
    #print (x, scale, compensation)

current_src1.set_status(1)
pna.set_bandwidth(500)
for control_id in [i for i in hivemind_qubit_controls.keys()]:
    channel_id = control_id
    compensated_setter = lambda x: set_control_compensated(x,  
                                                       lambda y:dac.set_voltage(channel_id, y),
                                                       current_src1.set_current,
                                                       hivemind_qubit_controls[channel_id]['Lr']*period_reference)
    sweep.sweep(pna, 
           (voltages, compensated_setter, 'Compensated voltage'), 
            filename='Hivemind #2 compensation test channel #{}'.format(channel_id))
    compensated_setter(0)

A Jupyter Widget

Started at:  May 16 2018 15:10:57


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


KeyboardInterrupt: 

KeyboardInterrupt: 

## Two-tone in resonator degeneracy point test

In [16]:
import pandas

In [16]:
dac=nndac()

In [17]:
## assembling indutance matrix for linear resonator drift compensation
LM0 = {25:{channel_id:hivemind_qubit_controls[channel_id]['Lr'] for channel_id in hivemind_qubit_controls.keys()}}
LM0.update({channel_id1:{channel_id2:1.0 if channel_id1==channel_id2 else 0.0 for channel_id2 in [25]+list(hivemind_qubit_controls.keys())} \
       for channel_id1 in hivemind_qubit_controls.keys()})
LM0[25][25]=1/period_reference

LM0_df = pandas.DataFrame(LM0)
LM0_inv = np.linalg.inv(np.asarray(LM0_df))
LM0_inv_df = LM0_df.copy()
LM0_inv_df.iloc[:,:] = LM0_inv.T
#LM0_inv_df

In [18]:
def set_control_resonator_max(cval, control_id, resonator_val=1/2.):
    target_vec = {_control_id:cval if _control_id == control_id else 0.0 for _control_id in hivemind_qubit_controls.keys()}
    target_vec[25] = centre_reference/period_reference+resonator_val # go to maximum
    target_fluxes = pandas.Series(target_vec)
    control_vec = np.asarray(np.dot(np.asarray(LM0_inv_df), np.asarray(target_fluxes))).ravel().tolist()
    column_names = [i for i in pandas.DataFrame(target_vec).columns]
    control_vec_df = pandas.DataFrame({n:v for n,v in zip(column_names, control_vec)})
    #return control_vec_df
    for channel_id in control_vec_df.keys():
        if channel_id<23:
            #print(type(control_vec_df[channel_id][0]), type(channel_id))
            dac.set_voltage(value=control_vec_df[channel_id][0], channel_number=channel_id)
        elif channel_id==25:
            current_src1.set_current(control_vec_df[channel_id])

In [20]:
set_control_resonator_max(-0.0, 0,-1/2)

In [413]:
pna.set_nop(101)
pna.set_xlim(9.9e9, 10.1e9)
pna.set_power(-50)
pna.set_bandwidth(1000)

True

In [183]:
dac = nndac()

In [21]:
lo1.set_status(0)
current_src1.set_status(1)
#pna.set_bandwidth(500)
voltages_highres = np.linspace(-4, 4, 101)
for control_id in [i for i in hivemind_qubit_controls.keys() if i<3]:
#for control_id in [2]:
    channel_id = control_id
    compensated_setter = lambda x: set_control_resonator_max(x, control_id, resonator_val=-1/2)
    pna.set_power(-15)
    pna.set_bandwidth(1000)
    sweep.sweep(pna, 
           (voltages_highres, compensated_setter, 'Compensated voltage #{}'.format(channel_id)), 
            filename='Hivemind #2 compensation test channel #{}'.format(channel_id))
    compensated_setter(0)

A Jupyter Widget

Started at:  May 21 2018 20:34:59

Elapsed time: 0 h 1 m 25.82 s


A Jupyter Widget

Started at:  May 21 2018 20:36:26

Elapsed time: 0 h 1 m 25.1 s


A Jupyter Widget

Started at:  May 21 2018 20:37:53

Elapsed time: 0 h 1 m 25.46 s


## Tone-tone 1d Scan (testing two-tone)

In [34]:
lo1.set_status(0)

True

In [20]:
pna.set_centerfreq(10.02e9)
pna.set_span(0)
pna.set_nop(1)
pna.set_bandwidth(5)
tone2_freqs = np.linspace(3e9, 8e9, 1001)
#tone2_powers = np.linspace(0, 18, 19)
lo1.set_power(10)
sweep.sweep(pna, 
            (tone2_freqs, lo1.set_frequency, 'Second tone frequency'), 
            filename='Two-tone frequency test scan power ex res-x ro res-z {}dBm'.format(int(lo1.get_power())))

A Jupyter Widget

Started at:  May 21 2018 13:00:06


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)



Elapsed time: 0 h 6 m 53.24 s


{'S-parameter': (('Second tone frequency',),
  (array([  3.00000000e+09,   3.00500000e+09,   3.01000000e+09, ...,
            7.99000000e+09,   7.99500000e+09,   8.00000000e+09]),),
  array([-0.00085865-0.0011788j , -0.00085080-0.00118373j,
         -0.00087402-0.00118511j, ..., -0.00091554-0.00114945j,
         -0.00090378-0.00115605j, -0.00093635-0.00114289j]),
  {'log': 20},
  ('',))}

In [21]:
pna.set_centerfreq(10.02e9)
pna.set_span(0)
pna.set_nop(1)
pna.set_bandwidth(5)
tone2_freqs = np.linspace(5.9e9, 6.3e9, 201)
tone2_powers = np.linspace(0, 18, 7)
#lo1.set_power(10)
sweep.sweep(pna, 
            (tone2_freqs, lo1.set_frequency, 'Second tone frequency'), 
            (tone2_powers, lo1.set_power, 'Second tone power'), 
            filename='Two-tone frequency test scan power ex antenna')

A Jupyter Widget

Started at:  May 21 2018 13:12:26


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)



Elapsed time: 0 h 10 m 42.18 s


{'S-parameter': (('Second tone frequency', 'Second tone power'),
  (array([  5.90000000e+09,   5.90200000e+09,   5.90400000e+09,
            5.90600000e+09,   5.90800000e+09,   5.91000000e+09,
            5.91200000e+09,   5.91400000e+09,   5.91600000e+09,
            5.91800000e+09,   5.92000000e+09,   5.92200000e+09,
            5.92400000e+09,   5.92600000e+09,   5.92800000e+09,
            5.93000000e+09,   5.93200000e+09,   5.93400000e+09,
            5.93600000e+09,   5.93800000e+09,   5.94000000e+09,
            5.94200000e+09,   5.94400000e+09,   5.94600000e+09,
            5.94800000e+09,   5.95000000e+09,   5.95200000e+09,
            5.95400000e+09,   5.95600000e+09,   5.95800000e+09,
            5.96000000e+09,   5.96200000e+09,   5.96400000e+09,
            5.96600000e+09,   5.96800000e+09,   5.97000000e+09,
            5.97200000e+09,   5.97400000e+09,   5.97600000e+09,
            5.97800000e+09,   5.98000000e+09,   5.98200000e+09,
            5.98400000e+09,   5.9860000

In [155]:
pna.set_xlim(6.5e9, 6.5e9)
pna.set_nop(1)

True

In [155]:

lo1.set_frequency(10e9)
lo1.set_status(1)
powers = np.linspace(-20, 18, 19)
sweep.sweep(pna, (powers, lo1.set_power, 'Second tone power'), filename='Check two-tone response presence on resonator freq')

A Jupyter Widget

Started at:  May 11 2018 22:57:44

Elapsed time: 0 h 0 m 12.17 s


{'S-parameter': (('Second tone power', 'Frequency'),
  (array([-20., -18., -16., -14., -12., -10.,  -8.,  -6.,  -4.,  -2.,   0.]),
   array([  6.50000000e+09,   6.51000000e+09,   6.52000000e+09,
            6.53000000e+09,   6.54000000e+09,   6.55000000e+09,
            6.56000000e+09,   6.57000000e+09,   6.58000000e+09,
            6.59000000e+09,   6.60000000e+09,   6.61000000e+09,
            6.62000000e+09,   6.63000000e+09,   6.64000000e+09,
            6.65000000e+09,   6.66000000e+09,   6.67000000e+09,
            6.68000000e+09,   6.69000000e+09,   6.70000000e+09,
            6.71000000e+09,   6.72000000e+09,   6.73000000e+09,
            6.74000000e+09,   6.75000000e+09,   6.76000000e+09,
            6.77000000e+09,   6.78000000e+09,   6.79000000e+09,
            6.80000000e+09,   6.81000000e+09,   6.82000000e+09,
            6.83000000e+09,   6.84000000e+09,   6.85000000e+09,
            6.86000000e+09,   6.87000000e+09,   6.88000000e+09,
            6.89000000e+09,   6.90000

## Two-tone current-frequency spectra

In [429]:
pna.set_centerfreq(9.975e9)
pna.set_nop(101)
pna.set_bandwidth(100)
pna.set_span(100e6)
plt.plot(pna.get_freqpoints(), pna.get_tracedata()[0])

In [427]:
lo1.set_status(1)
lo1.set_power(5)

True

In [152]:
set_control_resonator_max(-0.0, 0)

In [25]:
#pna.set_xlim(9.479e9, 9.479e9)
#pna.set_nop(1)
#pna.set_power(-50)
#lo1.set_power(5)
freqs = np.linspace(5.3e9, 6.3e9, 51)
voltages = np.linspace(-4, 4, 9)
pna.set_centerfreq(9.669e9)
pna.set_span(0)
pna.set_nop(1)
pna.set_bandwidth(5)
try:
    for channel_id in [i for i in hivemind_qubit_controls.keys() if i<3]:
        voltage_setter = lambda x: set_control_resonator_max(x, channel_id, resonator_val=-0.5);
        sweep.sweep(pna, 
            (voltages, voltage_setter, 'Compensated dac voltage #{0}'.format(channel_id)),
            (freqs, lo1.set_frequency, 'Second tone frequncy'), 
            filename='Hivemind two-tone res2 res-x ex, re-z ro -50 dbm #{0}'.format(channel_id))
finally:
    set_control_resonator_max(-0.0, 0)

A Jupyter Widget

Started at:  May 21 2018 21:00:42

Elapsed time: 0 h 3 m 16.52 s


A Jupyter Widget

Started at:  May 21 2018 21:04:00

Elapsed time: 0 h 3 m 14.91 s


A Jupyter Widget

Started at:  May 21 2018 21:07:17


KeyboardInterrupt: 

In [434]:
#pna.set_xlim(9.479e9, 9.479e9)
#pna.set_nop(1)
#pna.set_power(-50)
#lo1.set_power(5)
freqs = np.linspace(5e9, 8e9, 301)
voltages = np.linspace(-4, 4, 21)
pna.set_centerfreq(9.669e9)
pna.set_span(0)
pna.set_nop(1)
pna.set_bandwidth(5)
try:
    for channel_id in [i for i in hivemind_qubit_controls.keys() if i<3]:
        voltage_setter = lambda x: set_control_resonator_max(x, channel_id, resonator_val=0.5);
        sweep.sweep(pna, 
            (voltages, voltage_setter, 'Compensated dac voltage #{0}'.format(channel_id)),
            (freqs, lo1.set_frequency, 'Second tone frequncy'), 
            filename='Hivemind two-tone res2 res-x ex, re-z ro -50 dbm #{0}'.format(channel_id))
finally:
    set_control_resonator_max(-0.0, 0)

A Jupyter Widget

Started at:  May 16 2018 19:44:11


KeyboardInterrupt: 

KeyboardInterrupt: 

In [24]:
#pna.set_xlim(9.479e9, 9.479e9)
#pna.set_nop(1)
#pna.set_power(-50)
lo1.set_power(10)
freqs = np.linspace(6.15e9, 6.25e9, 51)
voltages = np.linspace(-4, 4, 21)
pna.set_centerfreq(10.02e9)
pna.set_span(0)
pna.set_nop(1)
pna.set_bandwidth(5)
#try:
for channel_id in [12,13,14,15,16,17,18,19,20,21,22,23]:
    voltage_setter = lambda x: dac.set_voltage(channel_id, x);
    sweep.sweep(pna, 
            (voltages, voltage_setter, 'Compensated dac voltage #{0}'.format(channel_id)),
            (freqs, lo1.set_frequency, 'Second tone frequncy'), 
            filename='Hivemind two-tone res2 res-x ex, re-z ro -50 dbm #{0}'.format(channel_id))
#finally:
#    pass
    dac.set_voltage(channel_id, 0.0);

A Jupyter Widget

Started at:  May 21 2018 13:27:09

Elapsed time: 0 h 7 m 51.24 s


A Jupyter Widget

Started at:  May 21 2018 13:35:01


KeyboardInterrupt: 

In [ ]:
inv
def set_control_with_mutual(x, target, compensator, scale):
    compensation = -x*scale
    target(x)
    compensator(compensation)
    #print (x, scale, compensation)

In [187]:
plt.figure('S21d')
plt.pcolormesh(np.unwrap(np.angle(S21d)))
plt.colorbar()
plt.figure('S21u')
plt.pcolormesh(np.real(inp))
plt.colorbar()
plt.figure('S21dr')
plt.pcolormesh(np.real(S21d))
plt.colorbar()

In [166]:
plt.figure()
plt.plot(np.arange(401), f)
plt.plot(np.arange(401)-period, f)

In [130]:
np.rollaxis?

In [51]:
f[np.isnan(f)]=0
plt.figure()
plt.plot(np.correlate(f-np.mean(f), f-np.mean(f), mode='full'))

d:\qtlab\ReiData\data/2018-05-10/15-15-07-Hivemind coil res-z\S-parameter Hivemind coil res-z.pkl


In [9]:
f = diff_resonance_finder(meas['S-parameter'], threshold=0, diff_axis=0, reduce_axis=1, diff_type='diff', min_separation=1)
plt.plot(meas['S-parameter'][1][0], f)

In [11]:
mdiff = diff_avg(meas, diff_axis=0)
plotting.plot_measurement(mdiff)#['S-parameter diff avg']

{'S-parameter amplitude': {'axes': <matplotlib.axes._axes.Axes at 0x218ef9789e8>,
  'plots': {'S-parameter': {'filter': <function plotting.plot_measurement.<locals>.<lambda>>,
    'mname': 'S-parameter',
    'plot': <matplotlib.image.AxesImage at 0x218ed7ca400>}}},
 'S-parameter diff avg amplitude': {'axes': <matplotlib.axes._axes.Axes at 0x218ed5e5978>,
  'plots': {'S-parameter diff avg': {'filter': <function plotting.plot_measurement.<locals>.<lambda>>,
    'mname': 'S-parameter diff avg',
    'plot': <matplotlib.image.AxesImage at 0x218ee0bcfd0>}}},
 'S-parameter diff avg phase': {'axes': <matplotlib.axes._axes.Axes at 0x218ef98f668>,
  'plots': {'S-parameter diff avg': {'filter': <function numpy.lib.function_base.angle>,
    'mname': 'S-parameter diff avg',
    'plot': <matplotlib.image.AxesImage at 0x218effebe48>}}},
 'S-parameter phase': {'axes': <matplotlib.axes._axes.Axes at 0x218ed5af940>,
  'plots': {'S-parameter': {'filter': <function numpy.lib.function_base.angle>,
    'mna

In [14]:
set_current_compensated(0.4e-3)

I = 0.0004
phase delta = 0.34042553191489366
voltage phase = -0.934971531315553
-0.934971531315553


In [66]:
currents = np.linspace(0.4e-3, -0.4e-3, 41)
current_src.set_status(1)
current_src2.set_status(1)
lo1.set_status(0)
pna.set_nop(101)
#pna.set_xlim(10.035e9,10.045e9)
pna.set_xlim(10.040e9, 10.050e9)

pna.set_bandwidth(100)
pna.set_power(-40)
sweep.sweep(pna, (currents, set_current_compensated, 'Coil current'), filename='Hivemind spectrum coil compensated')

A Jupyter Widget

Started at:  Feb 07 2018 18:19:12
I1 = 0.0004
I2 = -0.01271
I1 = 0.00038
I2 = -0.0120745
I1 = 0.00036
I2 = -0.011439
I1 = 0.00034
I2 = -0.0108035
I1 = 0.00032
I2 = -0.010168
I1 = 0.0003
I2 = -0.0095325
I1 = 0.00028
I2 = -0.008897
I1 = 0.00026
I2 = -0.0082615
I1 = 0.00024
I2 = -0.007626
I1 = 0.00022
I2 = -0.0069905
I1 = 0.0002
I2 = -0.006355
I1 = 0.00018
I2 = -0.0057195
I1 = 0.00016
I2 = -0.005084
I1 = 0.00014
I2 = -0.0044485
I1 = 0.00012
I2 = -0.003813
I1 = 0.0001
I2 = -0.0031775
I1 = 8e-05
I2 = -0.002542
I1 = 6e-05
I2 = -0.0019065
I1 = 4e-05
I2 = -0.001271
I1 = 2e-05
I2 = -0.0006355
I1 = 0.0
I2 = -0.0
I1 = -2e-05
I2 = 0.0006355
I1 = -4e-05
I2 = 0.001271
I1 = -6e-05
I2 = 0.0019065
I1 = -8e-05
I2 = 0.002542
I1 = -0.0001
I2 = 0.0031775
I1 = -0.00012
I2 = 0.003813
I1 = -0.00014
I2 = 0.0044485
I1 = -0.00016
I2 = 0.005084
I1 = -0.00018
I2 = 0.0057195
I1 = -0.0002
I2 = 0.006355
I1 = -0.00022
I2 = 0.0069905
I1 = -0.00024
I2 = 0.007626
I1 = -0.00026
I2 = 0.0082615
I1 = -0.00028
I2 = 0.008897
I

{'S-parameter': (('Coil current', 'Frequency'),
  (array([  4.00000000e-04,   3.80000000e-04,   3.60000000e-04,
            3.40000000e-04,   3.20000000e-04,   3.00000000e-04,
            2.80000000e-04,   2.60000000e-04,   2.40000000e-04,
            2.20000000e-04,   2.00000000e-04,   1.80000000e-04,
            1.60000000e-04,   1.40000000e-04,   1.20000000e-04,
            1.00000000e-04,   8.00000000e-05,   6.00000000e-05,
            4.00000000e-05,   2.00000000e-05,   0.00000000e+00,
           -2.00000000e-05,  -4.00000000e-05,  -6.00000000e-05,
           -8.00000000e-05,  -1.00000000e-04,  -1.20000000e-04,
           -1.40000000e-04,  -1.60000000e-04,  -1.80000000e-04,
           -2.00000000e-04,  -2.20000000e-04,  -2.40000000e-04,
           -2.60000000e-04,  -2.80000000e-04,  -3.00000000e-04,
           -3.20000000e-04,  -3.40000000e-04,  -3.60000000e-04,
           -3.80000000e-04,  -4.00000000e-04]),
   array([  1.00400000e+10,   1.00401000e+10,   1.00402000e+10,
        

In [102]:
def set_current_and_min_S21(x):
    set_current_compensated(x)
    lo1.set_status(0)
    pna.set_nop(121)
    pna.set_xlim(10.035e9, 10.05e9)
    pna.set_power(-45)
    pna.set_bandwidth(5)
    centerfreq = pna.get_freqpoints()[np.argmin(pna.get_tracedata()[0])]
    pna.set_xlim(centerfreq, centerfreq)
    pna.set_nop(1)
    pna.set_bandwidth(2)
    lo1.set_status(1)
    print ('Current:', x, ' frequency:', centerfreq)

In [113]:
#pna.set_xlim(9.479e9, 9.479e9)
pna.set_xlim(10.042e9, 10.042e9)
pna.set_nop(1)
pna.set_power(-45)
pna.set_bandwidth(1)
pna.set_average(1)
pna.set_averages(2)
lo1.set_status(1)
lo1.set_power(2)
#freqs = np.linspace(4e9, 8e9, 1601)
freqs = np.linspace(6.5e9, 8.5e9, 201)
#voltages = np.linspace(-4, 4, 41)
currents = np.linspace(-0.3e-3, -0.15e-3, 31)
sweep.sweep(pna, 
           (currents, set_current_and_min_S21, 'Coil current'), 
             (freqs, lo1.set_frequency, 'Second tone frequncy'), 
            filename='Hivemind two-tone coil-compensated passthrough')

A Jupyter Widget

Started at:  Feb 09 2018 14:06:13


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


I1 = -0.0003
I2 = 0.0095325
Current: -0.0003  frequency: 10042875000.0
I1 = -0.000295
I2 = 0.009373625
Current: -0.000295  frequency: 10042625000.0
I1 = -0.00029
I2 = 0.00921475
Current: -0.00029  frequency: 10042625000.0
I1 = -0.000285
I2 = 0.009055875
Current: -0.000285  frequency: 10042875000.0
I1 = -0.00028
I2 = 0.008897
Current: -0.00028  frequency: 10042375000.0
I1 = -0.000275
I2 = 0.008738125
Current: -0.000275  frequency: 10042625000.0
I1 = -0.00027
I2 = 0.00857925
Current: -0.00027  frequency: 10042500000.0
I1 = -0.000265
I2 = 0.008420375
Current: -0.000265  frequency: 10042875000.0
I1 = -0.00026
I2 = 0.0082615
Current: -0.00026  frequency: 10042625000.0
I1 = -0.000255
I2 = 0.008102625
Current: -0.000255  frequency: 10042625000.0
I1 = -0.00025
I2 = 0.00794375
Current: -0.00025  frequency: 10042625000.0
I1 = -0.000245
I2 = 0.007784875
Current: -0.000245  frequency: 10043000000.0
I1 = -0.00024
I2 = 0.007626
Current: -0.00024  frequency: 10042875000.0
I1 = -0.000235
I2 = 0.007467

{'S-parameter': (('Coil current', 'Second tone frequncy'),
  (array([-0.0003  , -0.000295, -0.00029 , -0.000285, -0.00028 , -0.000275,
          -0.00027 , -0.000265, -0.00026 , -0.000255, -0.00025 , -0.000245,
          -0.00024 , -0.000235, -0.00023 , -0.000225, -0.00022 , -0.000215,
          -0.00021 , -0.000205, -0.0002  , -0.000195, -0.00019 , -0.000185,
          -0.00018 , -0.000175, -0.00017 , -0.000165, -0.00016 , -0.000155,
          -0.00015 ]),
   array([  6.50000000e+09,   6.51000000e+09,   6.52000000e+09,
            6.53000000e+09,   6.54000000e+09,   6.55000000e+09,
            6.56000000e+09,   6.57000000e+09,   6.58000000e+09,
            6.59000000e+09,   6.60000000e+09,   6.61000000e+09,
            6.62000000e+09,   6.63000000e+09,   6.64000000e+09,
            6.65000000e+09,   6.66000000e+09,   6.67000000e+09,
            6.68000000e+09,   6.69000000e+09,   6.70000000e+09,
            6.71000000e+09,   6.72000000e+09,   6.73000000e+09,
            6.74000000e+09

In [109]:
#pna.set_xlim(9.479e9, 9.479e9)
pna.set_xlim(10.042e9, 10.042e9)
pna.set_nop(1)
pna.set_power(-45)
pna.set_average(1)
pna.set_averages(4)
pna.set_bandwidth(1)
lo1.set_status(1)
lo1.set_power(0)
#freqs = np.linspace(4e9, 8e9, 1601)
set_current_and_min_S21(-0.22e-3)
freqs = np.linspace(6.99e9, 7.00e9, 21)
powers = np.linspace(-10, 5, 6)
#voltages = np.linspace(-4, 4, 41)
#currents = np.linspace(-0.1e-3, 0.35e-3, 46)
sweep.sweep(pna, 
           (powers, lo1.set_power, 'Second tone power'), 
             (freqs, lo1.set_frequency, 'Second tone frequency'), 
            filename='Two-tone power scan')

I1 = -0.00022
I2 = 0.0069905
Current: -0.00022  frequency: 10042875000.0


A Jupyter Widget

Started at:  Feb 09 2018 12:14:48

Elapsed time: 0 h 4 m 11.65 s


{'S-parameter': (('Second tone power', 'Second tone frequency'),
  (array([-10.,  -7.,  -4.,  -1.,   2.,   5.]),
   array([  6.99000000e+09,   6.99050000e+09,   6.99100000e+09,
            6.99150000e+09,   6.99200000e+09,   6.99250000e+09,
            6.99300000e+09,   6.99350000e+09,   6.99400000e+09,
            6.99450000e+09,   6.99500000e+09,   6.99550000e+09,
            6.99600000e+09,   6.99650000e+09,   6.99700000e+09,
            6.99750000e+09,   6.99800000e+09,   6.99850000e+09,
            6.99900000e+09,   6.99950000e+09,   7.00000000e+09])),
  array([[-0.00136337 -1.44381518e-03j, -0.00118964 -1.40381046e-03j,
          -0.00099313 -1.47522823e-03j, -0.00099067 -1.34860841e-03j,
          -0.00138008 -1.27515104e-03j, -0.00128266 -1.41071435e-03j,
          -0.00103088 -1.29502150e-03j, -0.00120649 -1.27388840e-03j,
          -0.00140977 -1.19330816e-03j, -0.00121635 -1.14414864e-03j,
          -0.00132171 -1.34252897e-03j, -0.00114833 -1.14019995e-03j,
          -0.001

In [33]:
import imp
imp.reload(sweep)

<module 'sweep' from 'D:\\qtlab_replacement\\scripts\\sweep.py'>

In [120]:
#pna.set_xlim(9.479e9, 9.479e9)
pna.set_xlim(10.04e9, 10.04e9)
pna.set_nop(1)
pna.set_power(-35)
pna.set_average(0)
pna.set_averages(1)
pna.set_bandwidth(3)
lo1.set_status(1)
lo1.set_power(5)
freqs = np.linspace(6.5e9, 8e9,751)
#voltages = np.linspace(-4, 4, 41)
currents = np.linspace(-0.19e-3, -0.19e-3, 1)
#current = 0.05e-3
#set_current_and_min_S21(-0.22e-3)
for current in currents:
    #lo1.set_power(power)
    set_current_and_min_S21(current)
    sweep.sweep(pna, 
             (freqs, lo1.set_frequency, 'Second tone frequncy'), 
            filename='Hivemind two-tone coil-compensated power 0 dBm current'.format(current))

I1 = -0.00019
I2 = 0.00603725
Current: -0.00019  frequency: 10042875000.0


A Jupyter Widget

Started at:  Feb 09 2018 18:26:07


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


KeyboardInterrupt: 

In [9]:
set_res_z_voltage(-0)
current_src.set_current(0e-3)
current_src2.set_current(0)

True

### loading two-tone spectrum

In [117]:
filename = 'S-parameter Hivemind two-tone coil-compensated passthrough'
#location = 'D:\\qtlab\\ReiData\\data\\2018-02-02\\15-43-49-Hivemind two-tone coil-compensated'
#location = 'D:\\qtlab\\ReiData\\data\\2018-02-08\\11-05-12-Hivemind two-tone coil-compensated passthrough'
#location = 'D:\\qtlab\\ReiData\\data\\2017-05-24\\11-33-10'
location = 'D:\\qtlab\\ReiData\\data\\2018-02-09\\14-06-13-Hivemind two-tone coil-compensated passthrough'
data = load_pkl(filename, location=location)[1]
#D:\qtlab\ReiData\data\2018-02-02\15-43-49-Hivemind two-tone coil-compensated
plt.figure('Two-tone normalized weak', figsize=(18,6))
plt.subplot(1,2,1)
plt.pcolormesh(data['S-parameter'][1][0], 
               data['S-parameter'][1][1], 
               np.real(data['S-parameter'][2].T-np.mean(data['S-parameter'][2], axis=1)), cmap='RdBu_r')
plt.colorbar()
plt.subplot(1,2,2)
plt.pcolormesh(data['S-parameter'][1][0], 
               data['S-parameter'][1][1], 
               np.imag(data['S-parameter'][2].T-np.mean(data['S-parameter'][2], axis=1)), cmap='RdBu_r')
plt.colorbar()

D:\qtlab\ReiData\data\2018-02-09\14-06-13-Hivemind two-tone coil-compensated passthrough/S-parameter Hivemind two-tone coil-compensated passthrough.pkl


In [53]:
import fitting

### loading data for resonator frequency on coil current and fitting

In [44]:
filename = 'S-parameter Hivemind #2 single-tone'
#location = 'D:\\qtlab\\ReiData\\data\\2018-02-04\\11-57-19-Hivemind #2 single-tone'
#location = 'D:\\qtlab\\ReiData\\data\\2018-02-05\\10-44-59-Hivemind #2 single-tone'
#location = 'D:\\qtlab\\ReiData\\data\\2018-02-05\\10-52-24-Hivemind #2 single-tone'
location = 'D:\\qtlab\\ReiData\\data\\2018-02-05\\11-30-51-Hivemind #2 single-tone'
#location = 'D:\\qtlab\\ReiData\\data\\2017-05-24\\11-33-10'
data = load_pkl(filename, location=location)[1]

fitresults = fitting.dc_squid_fit_S21(data, noise_sigma=3)
L_res_coil = fitresults['fitresults']['L']
I0_coil = fitresults['fitresults']['I0']

D:\qtlab\ReiData\data\2018-02-05\11-30-51-Hivemind #2 single-tone/S-parameter Hivemind #2 single-tone.pkl


C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:779: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:427: RuntimeWarning: Number of calls to function has reached maxfev = 600.
  warnings.warn(errors[info][0], RuntimeWarning)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:427: RuntimeWarning: Number of calls to function has reached maxfev = 800.
  warnings.warn(errors[info][0], RuntimeWarning)
D:\qtlab_replacement\scripts\fitting.py:272: RuntimeWarning: invalid value encountered in greater
  fit_Qc[fit_Qc>1e5]=np.nan
D:\qtlab_replacem

In [28]:
plt.figure('fQl')
print(fitresults)
plt.scatter(fitresults['fit_fr'],fitresults['fit_Ql'], marker='o', linewidth=0, label='Ql')
plt.scatter(fitresults['fit_fr'],fitresults['fit_Qc'], marker='o', linewidth=0, label='Qc')
plt.scatter(fitresults['fit_fr'],fitresults['fit_Qi'], marker='o', linewidth=0, label='Qi')
plt.gca().set_yscale('log')
plt.xlabel('f, Hz')
plt.ylabel('Q')
plt.legend()
plt.ylim(1e1, 1e6)

{'fitresults': {'fp': nan, 'I0': nan, 'L': nan, 'a': nan, 'b': nan}, 'evaluator': <function dc_squid_fit_S21.<locals>.<lambda> at 0x000001D049A28598>, 'fp_est': array([  9.98300000e+09,   9.99100000e+09,   9.31700000e+09,
         1.00050000e+10,   1.00110000e+10,   1.00240000e+10,
         1.00270000e+10,   1.00310000e+10,   1.00340000e+10,
         1.00360000e+10,   1.00390000e+10,   1.00410000e+10,
         1.00420000e+10,   1.00430000e+10,   1.00440000e+10,
         1.00440000e+10,   1.00440000e+10,   9.31700000e+09,
         1.00420000e+10,   9.31700000e+09,   1.00380000e+10,
         1.00360000e+10,   1.00330000e+10,   1.00300000e+10,
         1.00260000e+10,   1.00230000e+10,   1.00100000e+10,
         9.31700000e+09,   9.31700000e+09,   9.31700000e+09,
         9.31700000e+09,   9.31700000e+09,   9.31700000e+09,
         9.85700000e+09,   9.85700000e+09,   9.85700000e+09,
         9.85600000e+09,   9.85600000e+09,   9.85400000e+09,
         9.85100000e+09,   9.87000000e+09,   9

(10.0, 1000000.0)

In [29]:
plt.pcolormesh(data['S-parameter'][1][0], 
               data['S-parameter'][1][1], 
               np.imag(data['S-parameter'][2].T-np.mean(data['S-parameter'][2], axis=1)), cmap='RdBu_r')
plt.plot(fitresults['fp_est_xpoints'], fitresults['fp_est'])

In [46]:
plt.figure()
plt.pcolormesh(data['S-parameter'][1][0], 
               data['S-parameter'][1][1], 
               np.abs(data['S-parameter'][2]-np.mean(data['S-parameter'][2], axis=0)).T, cmap='RdBu_r')
plt.grid()

In [23]:
filename = 'S-parameter Hivemind spectrum res-z'
location = 'D:\\qtlab\\ReiData\\data\\2017-05-24\\20-47-06'
data = load_pkl(filename, location=location)[1]

fitresults = fitting.dc_squid_fit_S21(data, noise_sigma=3)
L_res_z = fitresults['fitresults']['L']

D:\qtlab\ReiData\data\2017-05-24\20-47-06/S-parameter Hivemind spectrum res-z.pkl


C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:427: RuntimeWarning: Number of calls to function has reached maxfev = 600.
  warnings.warn(errors[info][0], RuntimeWarning)
C:\ProgramData\Anaconda3\lib\site-packages\resonator_tools-0.1-py3.6.egg\resonator_tools\circuit.py:132: RuntimeWarning: invalid value encountered in sqrt
C:\ProgramData\Anaconda3\lib\site-packages\resonator_tools-0.1-py3.6.egg\resonator_tools\circuit.py:137: RuntimeWarning: invalid value encountered in sqrt
C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:779: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
D:\qtlab_replacement\scrip

### getting resonance frequency dependence on coil current

In [50]:
plt.figure('S21')
plt.pcolormesh(data['S-parameter'][1][0], data['S-parameter'][1][1], np.abs(data['S-parameter'][2].T))
plt.plot(fitresults['fp_est_xpoints'], fitresults['fp_est'])
#plt.plot(fitresults['fp_est_xpoints'], fitresults['fp_est'])
plt.plot(data['S-parameter'][1][0], fitresults['evaluator'](data['S-parameter'][1][0]))
plt.ylim([np.min(data['S-parameter'][1][1]), np.max(data['S-parameter'][1][1])])
fitresults['fitresults']

D:\qtlab_replacement\scripts\fitting.py:125: RuntimeWarning: invalid value encountered in sqrt
  ft = lambda I,fp,I0,L,a,b: np.real(fp*(1+b)*(1/(1/np.sqrt((1-a)*np.abs(np.cos((I-I0)*L).astype(np.complex))+a)+b)))


{'I0': -0.32358758108292035,
 'L': 2.8544584239999415,
 'a': -0.11171427474883659,
 'b': 4.8703036587757254,
 'fp': 9722002636.158638}

In [30]:
LM = np.asarray([[L_res_z, L_res_coil]])
res_z_compensation = lambda I_coil: -2*np.pi*(((I_coil*LM[0,1]/(2*np.pi)+0.5)%1.)-0.5)/LM[0,0]*2.5
#res_z_compensation = lambda I_coil: np.pi*(((I_coil*LM[0,1]/(np.pi)+0.5)%1.)-0.5)/LM[0,0]
offset_coil_current = 0
def set_coil_current_compensated(current_value):
    set_res_z_voltage(res_z_compensation(current_value))
    current_src.set_current(current_value+offset_coil_current)
    F = current_value*LM[0,1]/(2*np.pi)
    Fc = res_z_compensation(current_value)*LM[0,0]/(2*np.pi)
    #print ('coil current: {0},\n res-z current: {1},\n fluxoid num: {2},\n compensation fluxoid num: {3}'.format(current_value, res_z_compensation(current_value), F, Fc))
    
currents = np.linspace(-0.3e-4, 0.3e-4, 31)
pna.set_startfreq(9.66e9)
pna.set_stopfreq(9.74e9)
pna.set_bandwidth(30)
pna.set_nop(41)
pna.set_power(-65)
sweep.sweep(pna, \
            (currents, set_coil_current_compensated, 'Resonator shift compensated coil current'), \
            filename='Hivemind spectrum res-z')

NameError: name 'L_res_z' is not defined

In [133]:
set_coil_current_compensated(0)

# Two-tone spectroscopy at 0 resonator control current

In [10]:
pna.set_xlim(9.723e9, 9.723e9)
pna.set_bandwidth(10)
pna.set_averages(1)
pna.set_nop(1)
pna.set_power(-60)
lo1.set_power(-10)
lo1.set_status(1)

ex_freqs = np.linspace(7.55e9, 7.6e9, 51)
currents = np.linspace(-3e-5, 3e-5, 61)
sweep.sweep(pna, (currents, set_coil_current_compensated, 'Qubit control 1 current'),\
                 (ex_freqs, lo1.set_frequency, 'Excitation frequency'),\
                filename='Hivemind 2T spectrum Q1')

Started at:  May 25 2017 18:54:05


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 8 m 3.37 s,	parameter values: Qubit control 1 current: 1.5e-05,	Excitation frequency: 7.578e+09,	average cycle time: 0.61s				s	

KeyboardInterrupt: 

Time left: 0 h 7 m 15.08 s,	parameter values: Qubit control 1 current: 1.7e-05,	Excitation frequency: 7.558e+09,	average cycle time: 0.62s	

KeyboardInterrupt: 

In [15]:
pna.set_xlim(9.723e9, 9.723e9)
pna.set_bandwidth(2)
pna.set_averages(1)
pna.set_nop(1)
pna.set_power(-60)
lo1.set_power(-10)
lo1.set_status(1)
ex_powers = np.linspace(-20,-5, 31)

#ex_freqs = np.linspace(6e9, 8e9, 201)
ex_freqs = np.linspace(7.00e9, 7.750e9, 251)
set_coil_current_compensated(0)
sweep.sweep(pna, (ex_powers, lo1.set_power, 'Excitation power'),\
                 (ex_freqs, lo1.set_frequency, 'Excitation frequency'),\
                filename='Hivemind 2T power sweep')

Started at:  May 26 2017 09:47:17


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s,	parameter values: Excitation power:     -5,	Excitation frequency: 7.75e+09,	average cycle time: 0.92s	s			
Elapsed time: 1 h 59 m 31.68 s


{'S-parameter': (('Excitation power', 'Excitation frequency'),
  (array([-20. , -19.5, -19. , -18.5, -18. , -17.5, -17. , -16.5, -16. ,
          -15.5, -15. , -14.5, -14. , -13.5, -13. , -12.5, -12. , -11.5,
          -11. , -10.5, -10. ,  -9.5,  -9. ,  -8.5,  -8. ,  -7.5,  -7. ,
           -6.5,  -6. ,  -5.5,  -5. ]),
   array([  7.00000000e+09,   7.00300000e+09,   7.00600000e+09,
            7.00900000e+09,   7.01200000e+09,   7.01500000e+09,
            7.01800000e+09,   7.02100000e+09,   7.02400000e+09,
            7.02700000e+09,   7.03000000e+09,   7.03300000e+09,
            7.03600000e+09,   7.03900000e+09,   7.04200000e+09,
            7.04500000e+09,   7.04800000e+09,   7.05100000e+09,
            7.05400000e+09,   7.05700000e+09,   7.06000000e+09,
            7.06300000e+09,   7.06600000e+09,   7.06900000e+09,
            7.07200000e+09,   7.07500000e+09,   7.07800000e+09,
            7.08100000e+09,   7.08400000e+09,   7.08700000e+09,
            7.09000000e+09,   7.093000

In [19]:
pna.set_xlim(9.723e9, 9.723e9)
pna.set_bandwidth(1)
pna.set_averages(1)
pna.set_nop(1)
pna.set_power(-60)
lo1.set_power(-15)
lo1.set_status(1)

#ex_freqs = np.linspace(6e9, 8e9, 201)
ex_freqs = np.linspace(7.00e9, 7.750e9, 751)
set_coil_current_compensated(0)
sweep.sweep(pna, (ex_freqs, lo1.set_frequency, 'Excitation frequency'),\
                filename='Hivemind 2T power sweep')

Started at:  May 26 2017 19:37:21
First sweep...

c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s,	parameter values: Excitation frequency: 7.75e+09,	average cycle time:  1.2s	s			
Elapsed time: 0 h 15 m 24.45 s


{'S-parameter': (('Excitation frequency',),
  (array([  7.00000000e+09,   7.00100000e+09,   7.00200000e+09,
            7.00300000e+09,   7.00400000e+09,   7.00500000e+09,
            7.00600000e+09,   7.00700000e+09,   7.00800000e+09,
            7.00900000e+09,   7.01000000e+09,   7.01100000e+09,
            7.01200000e+09,   7.01300000e+09,   7.01400000e+09,
            7.01500000e+09,   7.01600000e+09,   7.01700000e+09,
            7.01800000e+09,   7.01900000e+09,   7.02000000e+09,
            7.02100000e+09,   7.02200000e+09,   7.02300000e+09,
            7.02400000e+09,   7.02500000e+09,   7.02600000e+09,
            7.02700000e+09,   7.02800000e+09,   7.02900000e+09,
            7.03000000e+09,   7.03100000e+09,   7.03200000e+09,
            7.03300000e+09,   7.03400000e+09,   7.03500000e+09,
            7.03600000e+09,   7.03700000e+09,   7.03800000e+09,
            7.03900000e+09,   7.04000000e+09,   7.04100000e+09,
            7.04200000e+09,   7.04300000e+09,   7.04400000e+

# Pulsed measurements

In [11]:
#Параметры кубитов.
Qubits = {1: {'Fr': 10.04e9,'F01': 6.817e9, 'I0': -0.94e-3, 'Vr': -0.14, 'Ispan': 0.,'P': 0,'Pex': -2,'Span': 200e6}}
qubit_id = 1

In [12]:
current_src.set_current(Qubits[qubit_id]['I0'])
set_res_z_voltage(Qubits[qubit_id]['Vr'])

-0.14


In [13]:
import awg_digital
import awg_iq
import data_reduce
import imp
import fitting
import plotting
import save_pkl
import tomography as tomography_new
import tomography_legacy as tomography
import sweep
imp.reload(awg_digital)
imp.reload(awg_iq)
imp.reload(data_reduce)
imp.reload(fitting)
imp.reload(plotting)
imp.reload(save_pkl)
imp.reload(tomography)
imp.reload(tomography_new)
imp.reload(sweep)

<module 'sweep' from 'D:\\qtlab_replacement\\scripts\\sweep.py'>

In [14]:
# Источник тока - в autorange
#current.set_autorange(1)
#Мощности гетеродинов, постоянные
#Мощность гетеродина для возбуждения 13-16 дБм
#lo_ex_pow = 14
#lo_ex_pow = 14
lo_ex = lo1
lo_ro = pna
lo_ex_pow = 10
#lo_ex = lo1
lo_ex.set_status(1)
#lo_ex.set_power(lo_ex_pow)
#pna как lo2 (excitation)
#Port1 - выход
#Мощность гетеродина для считывания 10-13 дБм, с учётом разветвителя 13-16 dBm
#lo_ro_pow = 16
lo_ro_pow = 16
lo_ro.set_power(lo_ro_pow)

#lo_ex = pna
pna.write("OUTP ON")
pna.write("SOUR1:POW1:MODE ON")
pna.write("SOUR1:POW2:MODE OFF")
pna.set_sweep_mode("CW")
lo_ex.set_power(lo_ex_pow)

True

In [15]:
# промежуточные частоты для гетеродинной схемы:
#ex_if = 125e6
ex_if = 97.076e6
ro_if = 75e6
# клоки генератора и оцифровщика
ex_clock = 1e9
ro_clock = 1e9

rep_rate = 20e3 # частота повторений эксперимента

awg_tek.stop()
awg_tek.set_clock(ex_clock) # клок всех авгшк
awg_tek.set_nop(ex_clock/rep_rate) # репрейт нужно задавать по=хорошему только на управляющей,
awg_tek.run()
# а вот длину сэмплов, которая очевидно то же самое, нужно задавать на всех авгшках.
# хорошо, что сейчас она только одна.

# миксеры подключены к AWG5014 через 6 дБ аттенюаторы. Ставим амплитуду 0.7 В на считывание, 0.4 В на возбуждение.
for channel in range(3,5):
    awg_tek.set_amplitude(0.2, channel=channel)
    awg_tek.set_offset(0.0, channel=channel)
for channel in range(1,3):
    awg_tek.set_amplitude(0.2, channel=channel)
    awg_tek.set_offset(0.0, channel=channel)
# Выходы 1, 2 тектроникс: возбуждение (кубитный тон) через 6 дБ аттенюатор
# Выходы 3, 4 тектроникс: считывание (резонаторный тон) через 6 дБ аттенюатор
iq_ex = awg_iq.awg_iq(awg_tek, awg_tek, 1, 2, lo_ex)
iq_ro = awg_iq.awg_iq(awg_tek, awg_tek, 3, 4, lo_ro)
iq_ex.set_if(ex_if)
iq_ro.set_if(ro_if)
iq_ex.set_sideband_id(-1)
iq_ro.set_sideband_id(-1)
iq_ex.set_frequency(Qubits[qubit_id]['F01'])
iq_ro.set_frequency(Qubits[qubit_id]['Fr'])

# К первому маркере первого канала тектроникса подсоединён триггер оцифровщика.
# Надо подавать какой-нибудь триггер, что-ли.
# А ещё нужно подавать клок. К сожалению наш оцифровщик не может в 1ГГц клок, ему нужен 500МГц клок.
# Будем подавать 500 МГц с цифрового выхода тектроникса.
awg_tek.set_marker1_low(-1, channel=1)
awg_tek.set_marker2_low(-1, channel=1)
awg_tek.set_marker1_high(2, channel=1)
awg_tek.set_marker2_high(2, channel=1)
awg_tek.set_marker1_low(-0.1, channel=2)
awg_tek.set_marker2_low(-0.1, channel=2) 
awg_tek.set_marker1_high(0.1, channel=2) # с клока хватит и такой амплитуды
awg_tek.set_marker2_high(0.1, channel=2) # с клока хватит и такой амплитуды

ro_trg = awg_digital.awg_digital(awg_tek, 1)
osc_trg = awg_digital.awg_digital(awg_tek, 5)
ro_adc_clock = awg_digital.awg_digital(awg_tek, 2) #
osc_adc_clock = awg_digital.awg_digital(awg_tek, 6) #

# тупо клок
ro_adc_clock.set_waveform(np.asarray([0,1]*int(ro_adc_clock.get_nop()/2), dtype=int))
osc_adc_clock.set_waveform(np.asarray([0,1]*int(ro_adc_clock.get_nop()/2), dtype=int))
awg_channels = {'iq_ex':iq_ex, 'iq_ro':iq_ro, 'ro_trg':ro_trg, 'osc_trg':osc_trg}
trg_length = 4e-9
pg = tomography_new.pulses(awg_channels)

root: WARNING  Tektronix_AWG5014 : Clock set to 1000000000.0. This is not fully functional yet. To avoid problems, it is better not to change the clock during operation


In [16]:
# настройки оцифровщика
adc.reset()            
adc.set_timeout(10000)
adc.set_clock(ro_clock)

#adc.set_spc_samplerate(smp_rate)
#External trigger
adc.set_trigger_ext0_level0(0) # logical 0 value to 200 mV
adc.set_trigger_ext0_level1(1300) # logical 1 value to 600 mV
adc.trigger_termination_50Ohm()
adc.trigger_mode_pos()
adc.set_trigger_ext0_pulsewidth(0)
adc.disable_trigger_output()
adc.select_channel01()
adc.set_multi_mode()
#adc.set_segmentsize()
adc.set_trigger_ORmask_tmask_ext0()
adc.set_trigger_ANDmask_tmask_ext0()
adc.set_trigger_ORmask_tmask_NO_ch0()
adc.set_trigger_ORmask_tmask_NO_ch1()
adc.set_trigger_ANDmask_tmask_NO_ch0()
adc.set_trigger_ANDmask_tmask_NO_ch1()
# усилитель на оцифровщике
adc.set_input_amp_ch0(100)
adc.set_input_amp_ch1(100)
adc.set_input_offset_ch0(0)
adc.set_input_offset_ch1(0)

#External clock
adc.set_reference_clock(adc.get_clock())

# "Измеритель средних прошедших импульсов"
# Просто усредняет всё по номеру сэмпла.
adc_reducer = data_reduce.data_reduce(adc)
adc_reducer.filters['Mean Voltage (AC)'] = data_reduce.mean_reducer_noavg(adc, 'Voltage', 0)
adc_reducer.filters['S21+'] = data_reduce.mean_reducer_freq(adc, 'Voltage', 0, iq_ro.get_if())
adc_reducer.filters['S21-'] = data_reduce.mean_reducer_freq(adc, 'Voltage', 0, -iq_ro.get_if())
# Этот измеритель мы как правило используем когда точек не слишком много и все результаты его жизнедеятельности как правило 
# выглядят как ломаные. Чтобы было красиво, давайте лучше сделаем точки (а кривые потом получим фитованые)
adc_reducer.extra_opts['scatter'] = True

In [17]:
#sa.set_ref(0)
iq_ex.get_calibration(sa)
iq_ro.get_calibration(sa)

D:/qtlab/ReiData/data/calibrations//IQ-if9.7e+07-rf6.8e+09-sb--1.pkl
D:/qtlab/ReiData/data/calibrations//IQ-if7.5e+07-rf1e+10-sb--1.pkl


{'I': (0.24815696162357001+0.58801948760299316j),
 'Q': (0.61533943731723295+0.45854018255237844j),
 'dc': (0.13744921750395528-0.18661081811660854j),
 'num_sidebands': 7,
 'score': -60.106520981841484}

In [18]:
def set_adc_nop(ro_adc_length):
    adc.stop()
    signal_size = int(np.ceil(8./7.*(ro_adc_length)*adc.get_clock()))
    nop = int( 2**np.ceil(np.log2(signal_size)) )
    posttrigger = nop*7/8
    print (nop)
    adc.set_nop(nop)
    adc.set_post_trigger(posttrigger)

In [65]:
 
# ro_amplitude = 0.5 Qubit #2 preliminary
# ro_amplitude = 0.3 Qubit #3 preliminary
ro_dac_amplitude = 0.2
ro_dac_length = 0.3e-6 # вообще считывающие импульсы такой длины? тогда что такое adc 
# Ответ: ro_dac_length - это длительность генерируемого прямоугольного импульса
# ro_adc_length - это минимальное время, в течении которого мы считываем этот импульс, когда он прошёл через криостат.
# В криостате импульс деформируется и становится длинее. Сколько времени хотите его интегрировать, столько и ставьте.
# Наверное, имеет смысл сюда поставить на 1-2 времени затухания фотона в резонатор больше, чем ro_dac_length
#ro_adc_length = ro_dac_length*3
#ro_adc_length=0.8e-6 use it for bench in the end
ro_adc_length=0.8e-6 #
#adc.set_nums(30000)
#adc.set_software_nums_multi(1)
adc.set_nums(20000)
set_adc_nop(ro_adc_length)
adc.set_nums(100000)
#ro_adc_length
#adc.set_nums

512


True

In [62]:
plt.figure()
sa.set_xlim(10.0e9, 10.1e9)
plt.plot(*sa.get_tracedata())

In [57]:
adc.stop()

In [20]:
lo1.set_status(0)
iq_ro.set_waveform(np.ones(iq_ex.get_nop())*0.04)
iq_ex.set_waveform(np.ones(iq_ex.get_nop())*0.00)
sweep.sweep(adc_reducer, 
            (np.linspace(10.03e9, 10.05e9, 41), iq_ro.set_uncal_frequency, 'First tone'), 
            filename='Single  tone spectrocopy with IQ')
lo1.set_status(1)

A Jupyter Widget

Started at:  Feb 06 2018 15:05:30


KeyboardInterrupt: 

KeyboardInterrupt: 

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


KeyboardInterrupt: 

In [66]:
lo1.set_status(1)
lo1.set_power(14)
#iq_ro.set_waveform(np.ones(iq_ex.get_nop())*0.1)
print (ro_dac_amplitude)
sequence = [ pg.p(None, 10e-6), 
                pg.p('ro_trg', trg_length, pg.rect, 1),
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]
pg.set_seq(sequence)
iq_ex.set_waveform(np.ones(iq_ex.get_nop())*1)
iq_ro.set_frequency(Qubits[qubit_id]['Fr'])
sweep.sweep(adc_reducer, 
            (np.linspace(6.5e9, 7.0e9, 101), iq_ex.set_uncal_frequency, 'Second tone'), 
            (np.linspace(0.06e-3, 0.06e-3, 1), set_current_compensated, 'Compensated coil current'), 
            filename='Two tone spectrocopy with IQ')
lo1.set_status(1)

0.2


D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:647: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]


A Jupyter Widget

Started at:  Feb 07 2018 11:51:57


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


I = 6e-05
phase delta = 0.0510638297872
voltage phase = -0.140245729697
-0.140245729697

Elapsed time: 0 h 12 m 29.04 s


True

In [69]:
lo1.set_status(1)
iq_ro.set_waveform(np.ones(iq_ex.get_nop())*0.1)
iq_ex.set_waveform(np.ones(iq_ex.get_nop())*0.5)
set_current_compensated(0.0)
sweep.sweep(adc_reducer, 
            (np.linspace(6.5e9, 7.0e9, 101), iq_ex.set_uncal_frequency, 'Second tone'), 
            filename='Two tone spectrocopy with IQ')
lo1.set_status(1)

I = 0.0
phase delta = 0.0
voltage phase = -0.0
-0.0


A Jupyter Widget

Started at:  Feb 07 2018 12:40:55


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)



Elapsed time: 0 h 12 m 24.88 s


True

In [31]:
def set_pulse_amplitude(x):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    pg.set_seq([pg.p('ro_trg', trg_length, pg.rect, 1), pg.p('iq_ro', ro_dac_length, pg.rect, x)])
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (np.linspace(0.0, 0.06, 21), set_pulse_amplitude, 'Readout amplitude'), filename='Readout pulse passthrough')

A Jupyter Widget

Started at:  Feb 06 2018 15:08:44


D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:647: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]



Elapsed time: 0 h 3 m 2.88 s


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [34]:
ex_amplitude = 1.0#смотрим на график в S21-phase ?? как жить что частота от этого зависит
pause_length = 16e-9#пауза между возб.импульсом и считывающим
lengths = np.linspace(0.0e-6, 1.0e-6,501)#время наблюдения осцилляций,для фитования частоты достатоно пары периодов
readout_begin = np.max(lengths)
lo1.set_status(True)
def set_ex_length(length):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    sequence = [pg.p(None, readout_begin-length), 
                pg.p('iq_ex', length, pg.rect, ex_amplitude), 
                pg.p(None, pause_length), 
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]
    pg.set_seq(sequence)
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (lengths, set_ex_length, 'Rabi pulse length', 's'), filename='Rabi')
measurement_fitted, fitted_parameters_rabi = fitting.S21pm_fit(measurement, fitting.exp_sin_fit)
annotation = 'Phase: {0:4.4g} rad, Freq: {1:4.4g}, Decay: {2:4.4g} s'.format(fitted_parameters_rabi['phase'], 
                                                                             fitted_parameters_rabi['freq'], 
                                                                             fitted_parameters_rabi['decay'])
save_pkl.save_pkl({'type':'Rabi'}, measurement_fitted,annotation=annotation)

A Jupyter Widget

Started at:  Feb 05 2018 22:33:36


D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:647: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]



Elapsed time: 1 h 12 m 55.59 s


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


IndexError: index 501 is out of bounds for axis 1 with size 501

In [54]:
ex_amplitude = 1.0
pi_pulse = 10e-6
pause_length = 16e-9

delays = np.linspace(0e-6, 5e-6, 51)#серия задержек после возбуждения перед считыванием состояния
readout_begin = np.max(delays)+pi_pulse#зачем он тут прибавляет, а потом в коде вычитает? 
# раньше смысл был в том, чтобы менять длительность возбуждающей последовательности, не сдвигая считывание. То есть
# дополнительные импульсы приделываются до считывания (котороене двигается), а не считывание приделывается к концу.
# Сейчас это ещё раз реализовано в коде, которые делает последовательности импульсов (pg.set_seq). Если вдруг у нас будет
# много считываний, то там тоже надо будет мутить какую-нибудь синхронизацию.
def set_delay(delay):
    #awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p(None, readout_begin-pi_pulse),
                pg.p('iq_ex', pi_pulse, pg.rect, ex_amplitude), 
                pg.p(None, delay), 
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]
    
    pg.set_seq(sequence)
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (delays, set_delay, 'delay'), filename='Decay')

measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_fit)
#save_pkl.save_pkl({'type':'Decay'}, measurement_fitted)
annotation = 'Decay: {0:4.6g} s'.format(fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Decay'}, measurement_fitted, annotation=annotation)

A Jupyter Widget

Started at:  Feb 07 2018 08:44:11


D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:647: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]



Elapsed time: 0 h 7 m 33.85 s


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


[5.0877050954502784e-06, -0.015348072949736568, -0.021264953524001345, 0.029394994080049107, -0.0019874236005549273, -0.015338968560182964, -0.02127525910765183, 0.02939247361225982, -0.001961495538446413]
